In [20]:
import sys
sys.path.append("..")

In [21]:
from modules import grid
from modules import expectimax
from modules import shape
from modules import sysvariables
import torch as tr
import numpy as np

In [22]:
sysvariables.NODES = 0
sysvariables.DATASET = []

In [23]:
tetris = grid.Grid(5,5)

In [24]:
shape = shape.Shape()

In [25]:
expectimax.expectimax(depth = 3,grid = tetris,shape = shape.O)

[(<modules.grid.Grid at 0x15f674160>,
  array([[1, 1],
         [1, 1]])),
 (<modules.grid.Grid at 0x15f674040>,
  array([[1, 1],
         [1, 1]])),
 (<modules.grid.Grid at 0x15f674310>,
  array([[1, 1],
         [1, 1]])),
 (<modules.grid.Grid at 0x15f674370>,
  array([[1, 1],
         [1, 1]]))]

In [26]:
print(sysvariables.NODES)

31680


In [27]:
len(sysvariables.DATASET)

7792

In [28]:
sysvariables.DATASET

[(array([[0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.]]),
  array([[1, 1],
         [1, 1]]),
  -42.0),
 (array([[0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 1., 1.],
         [1., 1., 0., 1., 1.]]),
  array([[1, 1],
         [1, 1]]),
  -42.0),
 (array([[0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.]]),
  array([[1, 1],
         [1, 1]]),
  -30.0),
 (array([[0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 1., 1.],
         [1., 1., 0., 1., 1.]]),
  array([[1, 1],
         [1, 1]]),
  -30.0),
 (array([[1., 1., 1., 1., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.]]),
  array([[1, 1, 1, 1]]),
  -58.

In [ ]:
a,b,c = sysvariables.DATASET[0]

In [ ]:
c

In [ ]:

dataset_state = []
dataset_utilities = []
dataset_shape =[]
for a,b,c in sysvariables.DATASET:
    dataset_state.append(tr.Tensor(a))
    dataset_shape.append(tr.Tensor(b))
    dataset_utilities.append(-1*c)
    


In [ ]:
training_examples = []
testing_examples = []
training_examples.append(dataset_state[:6234])
testing_examples.append(dataset_state[6234:])


In [ ]:
training_examples.append(dataset_shape[:6234])
testing_examples.append(dataset_shape[6234:])


In [ ]:
training_examples.append(dataset_shape[:6234])
testing_examples.append(dataset_shape[6234:])

In [ ]:
len(training_examples[2])

In [ ]:
training_examples

In [ ]:
class LinNet(tr.nn.Module):
    def __init__(self, size, hid_features):
        super(LinNet, self).__init__()
        self.to_hidden = tr.nn.Sequential((5*size)**2, hid_features)
        self.to_output = tr.nn.Sequential(hid_features, 1)
    def forward(self, x):
        h = tr.relu(self.to_hidden(x.reshape(x.shape[0],-1)))
        #y1 = tr.sigmoid(self.to_output(h))
        y = tr.tanh(self.to_output(y1))
        return y

In [ ]:
def batch_error(net, batch):
    states, shapes, utilities = batch
    #print(utilities)
    u = utilities.reshape(-1,1).float()
    #print(u)
    y = net(states,shapes)
    e = tr.sum((y - u)**2) / utilities.shape[0]
    return e

In [ ]:
if __name__ == "__main__":

    # whether to loop over individual training examples or batch them
    batched = True

    # Make the network and optimizer
    net = LinNet(size=1, hid_features=10)
    optimizer = tr.optim.SGD(net.parameters(), lr=0.5)

    # Convert the states and their minimax utilities to tensors
    
    states, shapes, utilities = training_examples
    training_batch = tr.stack(states), tr.stack(shapes), tr.tensor(utilities)

    states, shapes, utilities = testing_examples
    testing_batch = tr.stack(states), tr.stack(shapes), tr.tensor(utilities)

    # Run the gradient descent iterations
    curves = [], []
    for epoch in range(50000):
    
        # zero out the gradients for the next backward pass
        optimizer.zero_grad()

        # loop version (slow)
        if not batched:
 
            training_error, testing_error = 0, 0

            for n, example in enumerate(zip(*training_examples)):
                e = example_error(net, example)
                e.backward()
                training_error += e.item()
            training_error /= len(training_examples)

            with tr.no_grad(): # less computationally expensive
                for n, example in enumerate(zip(*testing_examples)):
                    e = example_error(net, example)
                    testing_error += e.item()
                testing_error /= len(testing_examples)

        # batch version (fast)
        if batched:
            e = batch_error(net, training_batch)
            e.backward()
            training_error = e.item()

            with tr.no_grad():
                e = batch_error(net, testing_batch)
                testing_error = e.item()

        # take the next optimization step
        optimizer.step()    
        
        # print/save training progress
        if epoch % 1000 == 0:
            print("%d: %f, %f" % (epoch, training_error, testing_error))
        curves[0].append(training_error)
        curves[1].append(testing_error)

In [ ]:
import matplotlib.pyplot as pt
# visualize learning curves on train/test data
pt.plot(curves[0], 'b-')
pt.plot(curves[1], 'r-')
pt.plot([0, len(curves[1])], 'g-')
pt.plot()
pt.legend(["Train","Test","Baseline"])
pt.show()

In [ ]:
test = [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 0.]]
test = tr.tensor(np.array(test))



In [ ]:
net(test)